In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.linear_model import LogisticRegression
print(tf.__version__)

In [ ]:
class MLP:
  def __init__(self, vocab_size, hidden_unit, number_classer):
    super(MLP, self).__init__()
    self.vocab_size = vocab_size  
    self.hidden_unit = hidden_unit  
    self.NUMBER_CLASSES = number_classer  

 # Build model and return loss function, label predict  
  def build_model(self):
    self.X = tf.placeholder(tf.float32, shape=[None, self.vocab_size])
    self.Y = tf.placeholder(tf.int32, shape=[None, ])

    # weights and bias are variables to be updated
    weights_1 = tf.get_variable(name='weights_input_hidden', 
                                shape=(self.vocab_size, self.hidden_unit),
                                initializer=tf.random_normal_initializer(2018))
    biases_1 = tf.get_variable(name='biases_input_hidden',
                               shape=(self.hidden_unit),
                               initializer=tf.random_normal_initializer(2018))
    weights_2 = tf.get_variable(name='weights_hidden_output',
                                shape=(self.hidden_unit, self.NUMBER_CLASSES),
                                initializer=tf.random_normal_initializer(2018))
    biases_2 = tf.get_variable(name='biases_hidden_output',
                               shape=(self.NUMBER_CLASSES),
                               initializer=tf.random_normal_initializer(2018))
    
    # Calculate value of hidden layer
    hidden_units = tf.sigmoid(tf.matmul(self.X, weights_1) + biases_1)
    # Calculate value of output
    logit = tf.matmul(hidden_units, weights_2) + biases_2
    # Convert Y to form one hot
    label_one_hot = tf.one_hot(
            indices=self.Y, depth=self.NUMBER_CLASSES, dtype=tf.float32)
    # Calculate loss with cross entropy
    loss = tf.nn.softmax_cross_entropy_with_logits(
            labels=label_one_hot, logits=logit)
    loss = tf.reduce_mean(loss)

    # Predict label from output
    predict_label = tf.squeeze(tf.argmax(tf.nn.softmax(logit), axis=1))

    return predict_label, loss

  # Set optimal algorithm
  def trainer(self, loss, learning_rate):
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    return train_op

In [ ]:
class DataReader(object):

  def __init__(self, path_file, batch_size, vocab_size):
    super(DataReader, self).__init__()
    self.path_file = path_file
    self.vocab_size = vocab_size
    self.batch_size = batch_size
    self.data = []
    self.label = []

    with open(path_file) as f:
      data = f.read().split("\n")
    for line in data:
      vector_line = [0.0 for _ in range(vocab_size)]
      feature = line.split("<fff>")
      self.label.append(int(feature[0]))
      for word in feature[2].split():
        vector_line[int(word.split(':')[0])] = float(word.split(':')[1])
        # vector line, value of index is tf-idf value
      self.data.append(vector_line)
   
    self.data = np.array(self.data)
    self.label = np.array(self.label)
  
    self.num_epoch = 0
    self.batch_id = 0
  
  def next_batch(self):
    start = self.batch_id * self.batch_size
    end = start + self.batch_size
    #if call next batch, batch_id +1
    self.batch_id += 1

    if ((end + self.batch_size) > len(self.data)):
      end = len(self.data)
      self.num_epoch += 1
      self.batch_id = 0
      indices = [index for index in range(len(self.data))]
      random.seed(2018)
      random.shuffle(indices)
      self.data, self.label = self.data[indices], self.label[indices]
    
    return self.data[start:end], self.label[start:end]

In [ ]:
def train(vocab_size, hidden_unit, number_classer, learning_rate, epoch, path_train, path_test):
  print("------load data train------")
  train_reader = DataReader(path_file=path_train, batch_size=128, vocab_size=vocab_size)
  print("------load data test------")
  test_reader = DataReader(path_file=path_test, batch_size=128, vocab_size=vocab_size)
  print("------load finish------")
  
  #creat model MLP
  mlp = MLP(vocab_size=vocab_size, hidden_unit=hidden_unit, number_classer=number_classer)
  predict_label, loss = mlp.build_model()
  train_opt = mlp.trainer(loss, learning_rate)

  index_epoch = []
  loss_of_epoch =[]
  accuracy_epoch =[]

  #creat session
  with tf.Session() as sess:
    epoch, MAX_EPOCH = 0, epoch
    sess.run(tf.global_variables_initializer())
    while epoch < MAX_EPOCH:
      loss_print = 0
      if train_reader.batch_id == 0:
        count =1
      while count != 0:
        train_data, train_label = train_reader.next_batch()
        count = train_reader.batch_id
        _, loss_, _ = sess.run([predict_label, loss, train_opt], feed_dict={
            mlp.X: train_data,
            mlp.Y: train_label})
        loss_print = loss_
      epoch += 1
      print("------ epoch {} ------- loss {}".format(epoch, loss_print))
      index_epoch.append(epoch)
      loss_of_epoch.append(loss_print)
      #get accuracy of epoch
      num_true_preds = 0
      test_plabels_eval = sess.run(predict_label,
                                   feed_dict={
                                       mlp.X: test_reader.data,
                                       mlp.Y: test_reader.label})
      matches = np.equal(test_plabels_eval, test_reader.label)
      num_true_preds = np.mean(matches)
      acc = num_true_preds / len(test_reader.data)
      accuracy_epoch.append(acc)
      print("------Accuracy------", acc)
    #save parameters
    trainable_variables = tf.trainable_variables()
    for variable in trainable_variables:
      save_paramenters(name=variable.name, value=variable.eval(), epoch=epoch)
  plt.plot(index_epoch, loss_of_epoch)
  plt.plot(index_epoch, accuracy_epoch)
  plt.show()

In [ ]:
def save_paramenters(name, value, epoch):
  filename = name.replace(':', '-') + '-epoch-{}.txt'.format(epoch)
  # if length value shape = 1 => bias
  # else => weight
  if(len(value.shape) == 1):
    string_form = ','.join([str(number) for number in value])
  else:
    string_form = '\n'.join([','.join([str(number) for number in value[
                                row]]) for row in range(value.shape[0])])
  with open('./save_paras/' + filename, 'w') as f:
    f.write(string_form)

In [ ]:
def restore_paramenters(name, epoch):
  filename = name.replace(':', '-') + '-epoch-{}.txt'.format(epoch)
  with open('./save_paras/' + filename, 'r') as f:
    list_line = f.read().split('\n')
  # if number of line = 1 => bias
  # else => weight
  if(len(list_line) == 1):
    value = [float(number) for number in list_line[0].split(',')]
  else:
    value = [[float(number) for number in list_line[index].split(',')]
                 for index in range(len(list_line))]
  return value

In [ ]:
def test_model(vocab_size, hidden_unit, number_classer, epoch, path_test):
  # function get accuracy model trained
  mlp = MLP(vocab_size=vocab_size, hidden_unit=hidden_unit, number_classer=number_classer)
  predict_label, loss = mlp.build_model()
  # read data test
  test_reader = DataReader(path_file=path_test, batch_size=50, vocab_size=vocab_size)
  with tf.Session() as sess:
    trainable_variables = tf.trainable_variables()
    for variable in trainable_variables:
      saved_value = restore_paramenters(variable.name, epoch=epoch)
      assign_op = variable.assign(saved_value)
      sess.run(assign_op)
    num_true_preds = 0
    # loop data test and predict
    test_data, test_label = test_reader.next_batch()
    test_plabels_eval = sess.run(predict_label, 
                                 feed_dict={
                                     mlp.X: test_data,
                                     mlp.Y: test_label})
    matches = np.equal(test_plabels_eval, test_label)
    # num_true_preds  is number of predict label true
    num_true_preds += np.sum(matches.astype(float))
    print('------Accuracy ', num_true_preds / len(test_reader.data))

In [ ]:
if __name__ == '__main__':
  train(vocab_size=14140, hidden_unit=10, number_classer=20, learning_rate=0.0001, epoch=5,
        path_train="/data_train_tf_idf.txt", path_test="/data_test_tf_idf.txt")
    # test_model(vocab_size=14140, hidden_unit=100, number_classer=20,
    # epoch=10, path_test="/data_tf_idf.txt")
